## Linear Algebra
A lot of the Data Science methods we will see in this tutorial require some understanding of linear algebra, and in this notebook we will focus on how Julia handles matrices, the types that exist, and how to call basic linear algebra tasks.

In [1]:
# some packages we will use
using LinearAlgebra
using SparseArrays
using Images
using MAT

LoadError: ArgumentError: Package Images [916415d5-f1e6-5110-898d-aaa5f9f070e0] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


We will get started with creating a random matrix.

In [2]:
A = rand(10,10); # created a random matrix of size 10-by-10
Atranspose = A' # matrix transpose
A = A*Atranspose; # matrix multiplication

In [3]:
@show A[11] == A[1,2];

A[11] == A[1, 2] = true


In [4]:
b = rand(10); #created a random vector of size 10
x = A\b; #x is the solutions to the linear system Ax=b
@show norm(A*x-b)
;

norm(A * x - b) = 1.2732281025718309e-12


A few things that are noteworthy: 
- `A` is a `Matrix` type, and `b` is a `Vector` type.
- The transpose function creates a matrix of type `Adjoint`.
- `\` is always the recommended way to solve a linear system. You almost never want to call the `inv` function

In [5]:
@show typeof(A)
@show typeof(b)
@show typeof(rand(1,10))
@show typeof(Atranspose)
;

typeof(A) = Matrix{Float64}
typeof(b) = Vector{Float64}
typeof(rand(1, 10)) = Matrix{Float64}
typeof(Atranspose) = Adjoint{Float64, Matrix{Float64}}


In [6]:
Matrix{Float64} == Array{Float64,2}

true

In [7]:
Vector{Float64} == Array{Float64,1}

true

In [8]:
Atranspose

10×10 adjoint(::Matrix{Float64}) with eltype Float64:
 0.899095   0.567108  0.7495    …  0.342416   0.900006    0.478088
 0.357385   0.542398  0.142717     0.186974   0.0105183   0.815349
 0.609909   0.226176  0.603221     0.517351   0.874417    0.525564
 0.24934    0.470222  0.215289     0.0368718  0.00414774  0.0520776
 0.89089    0.740446  0.827303     0.517351   0.118662    0.608627
 0.82993    0.883993  0.25841   …  0.0938996  0.873765    0.0213447
 0.840542   0.620233  0.191332     0.968538   0.356105    0.255747
 0.0631519  0.363387  0.202795     0.558146   0.498329    0.668233
 0.694006   0.912154  0.570507     0.580043   0.680721    0.473689
 0.574491   0.264694  0.881931     0.599961   0.650271    0.732514

`adjoint` in julia is a lazy adjoint -- often, we can easily perform Linear Algebra operations such as `A*A'` without actually transposing the matrix.

In [9]:
?adjoint

search: adjoint adjoint! Adjoint



```
A'
adjoint(A)
```

Lazy adjoint (conjugate transposition). Note that `adjoint` is applied recursively to elements.

For number types, `adjoint` returns the complex conjugate, and therefore it is equivalent to the identity function for real numbers.

This operation is intended for linear algebra usage - for general data manipulation see [`permutedims`](@ref Base.permutedims).

# Examples

```jldoctest
julia> A = [3+2im 9+2im; 8+7im  4+6im]
2×2 Matrix{Complex{Int64}}:
 3+2im  9+2im
 8+7im  4+6im

julia> adjoint(A)
2×2 adjoint(::Matrix{Complex{Int64}}) with eltype Complex{Int64}:
 3-2im  8-7im
 9-2im  4-6im

julia> x = [3, 4im]
2-element Vector{Complex{Int64}}:
 3 + 0im
 0 + 4im

julia> x'x
25 + 0im
```


In [10]:
Atranspose.parent

10×10 Matrix{Float64}:
 0.899095  0.357385   0.609909  0.24934     …  0.0631519  0.694006  0.574491
 0.567108  0.542398   0.226176  0.470222       0.363387   0.912154  0.264694
 0.7495    0.142717   0.603221  0.215289       0.202795   0.570507  0.881931
 0.690899  0.0266786  0.157873  0.771487       0.47062    0.51957   0.687684
 0.669825  0.272774   0.800388  0.976454       0.252502   0.583298  0.956372
 0.487581  0.306493   0.730844  0.835914    …  0.912366   0.444309  0.665927
 0.973518  0.953743   0.515818  0.267981       0.168941   0.200608  0.527866
 0.342416  0.186974   0.517351  0.0368718      0.558146   0.580043  0.599961
 0.900006  0.0105183  0.874417  0.00414774     0.498329   0.680721  0.650271
 0.478088  0.815349   0.525564  0.0520776      0.668233   0.473689  0.732514

In [11]:
sizeof(A)

800

That's because it's an array of Float64's, each is of size 8 bytes, and there are 10*10 numbers.

In [12]:
# To actually copy the matrix:
B = copy(Atranspose)

10×10 Matrix{Float64}:
 0.899095   0.567108  0.7495    …  0.342416   0.900006    0.478088
 0.357385   0.542398  0.142717     0.186974   0.0105183   0.815349
 0.609909   0.226176  0.603221     0.517351   0.874417    0.525564
 0.24934    0.470222  0.215289     0.0368718  0.00414774  0.0520776
 0.89089    0.740446  0.827303     0.517351   0.118662    0.608627
 0.82993    0.883993  0.25841   …  0.0938996  0.873765    0.0213447
 0.840542   0.620233  0.191332     0.968538   0.356105    0.255747
 0.0631519  0.363387  0.202795     0.558146   0.498329    0.668233
 0.694006   0.912154  0.570507     0.580043   0.680721    0.473689
 0.574491   0.264694  0.881931     0.599961   0.650271    0.732514

In [13]:
sizeof(B)

800

The `\` operator allows you to solve a system of linear equations, and often uses a suitable matrix factorization to solve the problem. We will cover factorizations next.

In [14]:
?\

search: \



```
\(x, y)
```

Left division operator: multiplication of `y` by the inverse of `x` on the left. Gives floating-point results for integer arguments.

# Examples

```jldoctest
julia> 3 \ 6
2.0

julia> inv(3) * 6
2.0

julia> A = [4 3; 2 1]; x = [5, 6];

julia> A \ x
2-element Vector{Float64}:
  6.5
 -7.0

julia> inv(A) * x
2-element Vector{Float64}:
  6.5
 -7.0
```

---

```
\(A, B)
```

Matrix division using a polyalgorithm. For input matrices `A` and `B`, the result `X` is such that `A*X == B` when `A` is square. The solver that is used depends upon the structure of `A`.  If `A` is upper or lower triangular (or diagonal), no factorization of `A` is required and the system is solved with either forward or backward substitution. For non-triangular square matrices, an LU factorization is used.

For rectangular `A` the result is the minimum-norm least squares solution computed by a pivoted QR factorization of `A` and a rank estimate of `A` based on the R factor.

When `A` is sparse, a similar polyalgorithm is used. For indefinite matrices, the `LDLt` factorization does not use pivoting during the numerical factorization and therefore the procedure can fail even for invertible matrices.

See also: [`factorize`](@ref), [`pinv`](@ref).

# Examples

```jldoctest
julia> A = [1 0; 1 -2]; B = [32; -4];

julia> X = A \ B
2-element Vector{Float64}:
 32.0
 18.0

julia> A * X == B
true
```

---

```
(\)(F::QRSparse, B::StridedVecOrMat)
```

Solve the least squares problem $\min\|Ax - b\|^2$ or the linear system of equations $Ax=b$ when `F` is the sparse QR factorization of $A$. A basic solution is returned when the problem is underdetermined.

# Examples

```jldoctest
julia> A = sparse([1,2,4], [1,1,1], [1.0,1.0,1.0], 4, 2)
4×2 SparseMatrixCSC{Float64, Int64} with 3 stored entries:
 1.0   ⋅
 1.0   ⋅
  ⋅    ⋅
 1.0   ⋅

julia> qr(A)\fill(1.0, 4)
2-element Vector{Float64}:
 1.0
 0.0
```


### 🟢Factorizations
A common tool used in Linear Algebra is matrix factorizations. These factorizations are often used to solve linear systems like `Ax=b`, and as we will see later in this tutorial... `Ax=b` comes up in a lot of Data Science problems

#### LU factorization
L\*U = P\*A

In [15]:
luA = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
10×10 Matrix{Float64}:
 1.0        0.0          0.0        0.0        …   0.0        0.0        0.0
 0.84153    1.0          0.0        0.0            0.0        0.0        0.0
 0.725545  -0.267172     1.0        0.0            0.0        0.0        0.0
 0.65913    0.458668     0.496967   1.0            0.0        0.0        0.0
 0.958321   0.0720283    0.828623   0.580649       0.0        0.0        0.0
 0.860511   0.708859     0.461131   0.788741   …   0.0        0.0        0.0
 0.875501   0.112092    -0.230278   0.0821518      0.0        0.0        0.0
 0.647973   0.00349253   0.270735   0.18293        1.0        0.0        0.0
 0.766866  -0.0969748    0.362346   0.325745      -0.106514   1.0        0.0
 0.599198   0.274241     0.943496  -0.298024       0.662348  -0.0165358  1.0
U factor:
10×10 Matrix{Float64}:
 4.37491  3.68161    3.17419   2.88363   4.19256   …   3.35497     2.62144
 0.0      0.538414  -0.143849  0.246953  0.038781     -0.052

In [16]:
norm(luA.L*luA.U - luA.P*A)

1.88411095042053e-15

#### QR factorization
Q\*R = A

In [17]:
qrA = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
10×10 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.393402   0.240188    0.404577    0.155726   …   0.297149    0.626583
 -0.33106   -0.650887    0.167787    0.379969      -0.189573   -0.276244
 -0.285431   0.402168   -0.411695    0.176959       0.458561   -0.452107
 -0.259303  -0.232935   -0.185937   -0.580353       0.0970939   0.177018
 -0.377006   0.168736   -0.247324   -0.225245      -0.528909   -0.000122666
 -0.338527  -0.397982   -0.12074    -0.309283   …   0.406587    0.00606434
 -0.344424   0.114669    0.507876    0.0633789      0.015783   -0.294585
 -0.254914   0.152656    0.0581301  -0.0205874      0.0194476  -0.251674
 -0.301687   0.266913    0.0547462  -0.138236      -0.44734     0.00629991
 -0.235726  -0.0900111  -0.513846    0.541579      -0.1013      0.380985
R factor:
10×10 Matrix{Float64}:
 -11.1207  -9.74376   -8.6678    -8.33006   …  -9.2051    -7.84364
   0.0     -0.631191   0.325737  -0.37795

In [18]:
norm(qrA.Q*qrA.R - A)

1.1512108063996359e-14

#### Cholesky factorization, note that A needs to be symmetric positive definite
L\*L' = A 

In [19]:
isposdef(A)

true

In [20]:
cholA = cholesky(A)

Cholesky{Float64, Matrix{Float64}}
U factor:
10×10 UpperTriangular{Float64, Matrix{Float64}}:
 2.09163  1.76017    1.51757   …   1.35532     1.604       1.2533
  ⋅       0.733767  -0.196042      0.0025627  -0.0711569   0.201229
  ⋅        ⋅         0.763618      0.206738    0.276694    0.720471
  ⋅        ⋅          ⋅            0.136803    0.243606   -0.222875
  ⋅        ⋅          ⋅            0.0960975  -0.473753    0.0842238
  ⋅        ⋅          ⋅        …   0.243754    0.601727    0.339648
  ⋅        ⋅          ⋅           -0.189585   -0.0894905   0.480567
  ⋅        ⋅          ⋅            0.800464   -0.0852608   0.530186
  ⋅        ⋅          ⋅             ⋅          0.545979   -0.00902822
  ⋅        ⋅          ⋅             ⋅           ⋅          0.0386228

In [21]:
norm(cholA.L*cholA.U - A)

1.7763568394002505e-15

In [22]:
cholA.L

10×10 LowerTriangular{Float64, Matrix{Float64}}:
 2.09163    ⋅           ⋅        …    ⋅           ⋅           ⋅ 
 1.76017   0.733767     ⋅             ⋅           ⋅           ⋅ 
 1.51757  -0.196042    0.763618       ⋅           ⋅           ⋅ 
 1.37865   0.336555    0.379493       ⋅           ⋅           ⋅ 
 2.00445   0.052852    0.632751       ⋅           ⋅           ⋅ 
 1.79987   0.520137    0.352128  …    ⋅           ⋅           ⋅ 
 1.83122   0.0822494  -0.175845       ⋅           ⋅           ⋅ 
 1.35532   0.0025627   0.206738      0.800464     ⋅           ⋅ 
 1.604    -0.0711569   0.276694     -0.0852608   0.545979     ⋅ 
 1.2533    0.201229    0.720471      0.530186   -0.00902822  0.0386228

In [23]:
cholA.U

10×10 UpperTriangular{Float64, Matrix{Float64}}:
 2.09163  1.76017    1.51757   …   1.35532     1.604       1.2533
  ⋅       0.733767  -0.196042      0.0025627  -0.0711569   0.201229
  ⋅        ⋅         0.763618      0.206738    0.276694    0.720471
  ⋅        ⋅          ⋅            0.136803    0.243606   -0.222875
  ⋅        ⋅          ⋅            0.0960975  -0.473753    0.0842238
  ⋅        ⋅          ⋅        …   0.243754    0.601727    0.339648
  ⋅        ⋅          ⋅           -0.189585   -0.0894905   0.480567
  ⋅        ⋅          ⋅            0.800464   -0.0852608   0.530186
  ⋅        ⋅          ⋅             ⋅          0.545979   -0.00902822
  ⋅        ⋅          ⋅             ⋅           ⋅          0.0386228

In [24]:
factorize(A)

Cholesky{Float64, Matrix{Float64}}
U factor:
10×10 UpperTriangular{Float64, Matrix{Float64}}:
 2.09163  1.76017    1.51757   …   1.35532     1.604       1.2533
  ⋅       0.733767  -0.196042      0.0025627  -0.0711569   0.201229
  ⋅        ⋅         0.763618      0.206738    0.276694    0.720471
  ⋅        ⋅          ⋅            0.136803    0.243606   -0.222875
  ⋅        ⋅          ⋅            0.0960975  -0.473753    0.0842238
  ⋅        ⋅          ⋅        …   0.243754    0.601727    0.339648
  ⋅        ⋅          ⋅           -0.189585   -0.0894905   0.480567
  ⋅        ⋅          ⋅            0.800464   -0.0852608   0.530186
  ⋅        ⋅          ⋅             ⋅          0.545979   -0.00902822
  ⋅        ⋅          ⋅             ⋅           ⋅          0.0386228

In [25]:
?factorize

search: factorize Factorization factorial



```
factorize(A)
```

Compute a convenient factorization of `A`, based upon the type of the input matrix. `factorize` checks `A` to see if it is symmetric/triangular/etc. if `A` is passed as a generic matrix. `factorize` checks every element of `A` to verify/rule out each property. It will short-circuit as soon as it can rule out symmetry/triangular structure. The return value can be reused for efficient solving of multiple systems. For example: `A=factorize(A); x=A\b; y=A\C`.

| Properties of `A`          | type of factorization                      |
|:-------------------------- |:------------------------------------------ |
| Positive-definite          | Cholesky (see [`cholesky`](@ref))          |
| Dense Symmetric/Hermitian  | Bunch-Kaufman (see [`bunchkaufman`](@ref)) |
| Sparse Symmetric/Hermitian | LDLt (see [`ldlt`](@ref))                  |
| Triangular                 | Triangular                                 |
| Diagonal                   | Diagonal                                   |
| Bidiagonal                 | Bidiagonal                                 |
| Tridiagonal                | LU (see [`lu`](@ref))                      |
| Symmetric real tridiagonal | LDLt (see [`ldlt`](@ref))                  |
| General square             | LU (see [`lu`](@ref))                      |
| General non-square         | QR (see [`qr`](@ref))                      |

If `factorize` is called on a Hermitian positive-definite matrix, for instance, then `factorize` will return a Cholesky factorization.

# Examples

```jldoctest
julia> A = Array(Bidiagonal(fill(1.0, (5, 5)), :U))
5×5 Matrix{Float64}:
 1.0  1.0  0.0  0.0  0.0
 0.0  1.0  1.0  0.0  0.0
 0.0  0.0  1.0  1.0  0.0
 0.0  0.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0

julia> factorize(A) # factorize will check to see that A is already factorized
5×5 Bidiagonal{Float64, Vector{Float64}}:
 1.0  1.0   ⋅    ⋅    ⋅
  ⋅   1.0  1.0   ⋅    ⋅
  ⋅    ⋅   1.0  1.0   ⋅
  ⋅    ⋅    ⋅   1.0  1.0
  ⋅    ⋅    ⋅    ⋅   1.0
```

This returns a `5×5 Bidiagonal{Float64}`, which can now be passed to other linear algebra functions (e.g. eigensolvers) which will use specialized methods for `Bidiagonal` types.


In [26]:
?diagm

search: diagm spdiagm diag diagind Diagonal isdiag Bidiagonal Tridiagonal



```
diagm(kv::Pair{<:Integer,<:AbstractVector}...)
diagm(m::Integer, n::Integer, kv::Pair{<:Integer,<:AbstractVector}...)
```

Construct a matrix from `Pair`s of diagonals and vectors. Vector `kv.second` will be placed on the `kv.first` diagonal. By default the matrix is square and its size is inferred from `kv`, but a non-square size `m`×`n` (padded with zeros as needed) can be specified by passing `m,n` as the first arguments.

`diagm` constructs a full matrix; if you want storage-efficient versions with fast arithmetic, see [`Diagonal`](@ref), [`Bidiagonal`](@ref) [`Tridiagonal`](@ref) and [`SymTridiagonal`](@ref).

# Examples

```jldoctest
julia> diagm(1 => [1,2,3])
4×4 Matrix{Int64}:
 0  1  0  0
 0  0  2  0
 0  0  0  3
 0  0  0  0

julia> diagm(1 => [1,2,3], -1 => [4,5])
4×4 Matrix{Int64}:
 0  1  0  0
 4  0  2  0
 0  5  0  3
 0  0  0  0
```

---

```
diagm(v::AbstractVector)
diagm(m::Integer, n::Integer, v::AbstractVector)
```

Construct a matrix with elements of the vector as diagonal elements. By default, the matrix is square and its size is given by `length(v)`, but a non-square size `m`×`n` can be specified by passing `m,n` as the first arguments.

# Examples

```jldoctest
julia> diagm([1,2,3])
3×3 Matrix{Int64}:
 1  0  0
 0  2  0
 0  0  3
```


In [27]:
# convert(Diagonal{Int64,Array{Int64,1}},diagm([1,2,3]))
Diagonal([1,2,3])

3×3 Diagonal{Int64, Vector{Int64}}:
 1  ⋅  ⋅
 ⋅  2  ⋅
 ⋅  ⋅  3

`I` is a function

In [28]:
I(3)

3×3 Diagonal{Bool, Vector{Bool}}:
 1  ⋅  ⋅
 ⋅  1  ⋅
 ⋅  ⋅  1

### 🟢Sparse Linear Algebra
Sparse matrices are stored in Compressed Sparse Column (CSC) form

In [29]:
using SparseArrays
S = sprand(5,5,2/5)

5×5 SparseMatrixCSC{Float64, Int64} with 15 stored entries:
  ⋅        0.88834      ⋅         ⋅         ⋅ 
 0.201926  0.338059     ⋅        0.606466  0.651857
  ⋅        0.00870446  0.773248  0.166361  0.233344
  ⋅        0.706706    0.743204   ⋅         ⋅ 
 0.463766  0.3396      0.86463   0.561428   ⋅ 

In [30]:
S.rowval

15-element Vector{Int64}:
 2
 5
 1
 2
 3
 4
 5
 3
 4
 5
 2
 3
 5
 2
 3

In [31]:
Matrix(S)

5×5 Matrix{Float64}:
 0.0       0.88834     0.0       0.0       0.0
 0.201926  0.338059    0.0       0.606466  0.651857
 0.0       0.00870446  0.773248  0.166361  0.233344
 0.0       0.706706    0.743204  0.0       0.0
 0.463766  0.3396      0.86463   0.561428  0.0

In [32]:
S.colptr

6-element Vector{Int64}:
  1
  3
  8
 11
 14
 16

In [33]:
S.m

5

### 🟢Images as matrices
Let's get to the more "data science-y" side. We will do so by working with images (which can be viewed as matrices), and we will use the `SVD` decomposition.

First let's load an image. I chose this image as it has a lot of details.

In [34]:
X1 = load("data/khiam-small.jpg")

LoadError: UndefVarError: load not defined

In [35]:
@show typeof(X1)
X1[1,1] # this is pixel [1,1]

LoadError: UndefVarError: X1 not defined

We can easily convert this image to gray scale.

In [36]:
Xgray = Gray.(X1)

LoadError: UndefVarError: Gray not defined

We can easily extract the RGB layers from the image. We will make use of the `reshape` function below to reshape a vector to a matrix.

In [37]:
R = map(i->X1[i].r,1:length(X1))
R = Float64.(reshape(R,size(X1)...))

G = map(i->X1[i].g,1:length(X1))
G = Float64.(reshape(G,size(X1)...))

B = map(i->X1[i].b,1:length(X1))
B = Float64.(reshape(B,size(X1)...))
;

LoadError: UndefVarError: X1 not defined

In [38]:
Z = zeros(size(R)...) # just a matrix of all zeros of equal size as the image
RGB.(Z,G,Z)

LoadError: UndefVarError: R not defined

We can easily obtain the `Float64` values of the grayscale image.

In [39]:
Xgrayvalues = Float64.(Xgray)

LoadError: UndefVarError: Xgray not defined

Next, we will downsample this image using the SVD. First, let's obtain the SVD decomposition.

In [40]:
SVD_V = svd(Xgrayvalues)

LoadError: UndefVarError: Xgrayvalues not defined

In [41]:
norm(SVD_V.U*diagm(SVD_V.S)*SVD_V.V' - Xgrayvalues)

LoadError: UndefVarError: SVD_V not defined

In [42]:
# use the top 4 singular vectors/values to form a new image
u1 = SVD_V.U[:,1]
v1 = SVD_V.V[:,1]
img1 = SVD_V.S[1]*u1*v1'

i = 2
u1 = SVD_V.U[:,i]
v1 = SVD_V.V[:,i]
img1 += SVD_V.S[i]*u1*v1'

i = 3
u1 = SVD_V.U[:,i]
v1 = SVD_V.V[:,i]
img1 += SVD_V.S[i]*u1*v1'

i = 4
u1 = SVD_V.U[:,i]
v1 = SVD_V.V[:,i]
img1 += SVD_V.S[i]*u1*v1'

LoadError: UndefVarError: SVD_V not defined

In [43]:
Gray.(img1)

LoadError: UndefVarError: Gray not defined

As you can see, it's still far away from the original image. Let's try using 100 singular vectors/values.

In [44]:
i = 1:100
u1 = SVD_V.U[:,i]
v1 = SVD_V.V[:,i]
img1 = u1*spdiagm(0=>SVD_V.S[i])*v1'
Gray.(img1)

LoadError: UndefVarError: SVD_V not defined

This looks almost identical to the original image, even though it's not identical to the original image (and we can see that from the norm difference).

In [45]:
norm(Xgrayvalues-img1)

LoadError: UndefVarError: Xgrayvalues not defined

Our next problem will still be related to images, but this time we will solve a simple form of the face recognition problem. Let's get the data first.

In [46]:
M = matread("data/face_recog_qr.mat")

LoadError: UndefVarError: matread not defined

Each vector in `M["V2"]` is a fase image. Let's reshape the first one and take a look.

In [47]:
q = reshape(M["V2"][:,1],192,168)
Gray.(q)

LoadError: UndefVarError: M not defined

Now we will go back to the vectorized version of this image, and try to select the images that are most similar to it from the "dictionary" matrix. Let's use `b = q[:]` to be the query image. Note that the notation `[:]` vectorizes a matrix column wise.

In [48]:
b = q[:]

LoadError: UndefVarError: q not defined

We will remove the first image from the dictionary. The goal is to find the solution of the linear system `Ax=b` where `A` is the dictionary of all images. In face recognition problem we really want to minimize the norm differece `norm(Ax-b)` but the `\` actually solves a least squares problem when the matrix at hand is not invertible.

In [49]:
A = M["V2"][:,2:end]
x = A\b #Ax=b
Gray.(reshape(A*x,192,168))

LoadError: UndefVarError: M not defined

In [50]:
norm(A*x-b)

1.2732281025718309e-12

This was an easy problem. Let's try to make the picture harder to recover. We will add some random error.

In [51]:
qv = q+rand(size(q,1),size(q,2))*0.5
qv = qv./maximum(qv)
Gray.(qv)

LoadError: UndefVarError: q not defined

In [52]:
b = qv[:];

LoadError: UndefVarError: qv not defined

In [53]:
x = A\b
norm(A*x-b)

1.2732281025718309e-12

The error is so much bigger this time.

In [54]:
Gray.(reshape(A*x,192,168))

LoadError: DimensionMismatch("new dimensions (192, 168) must be consistent with array size 10")

# Finally...
After finishing this notebook, you should be able to:
- [ ] reshape and vectorize a matrix
- [ ] apply basic linear algebra operations such as transpose, matrix-matrix product, and solve a linear systerm
- [ ] call a linear algebra factorization on your matrix
- [ ] use SVD to created a compressed version of an image
- [ ] solve the face recognition problem via a least square approach
- [ ] create a sparse matrix, and call the components of the Compressed Sparse Column storage
- [ ] list a few types of matrices Julia uses (diagonal, upper triangular,...)
- [ ] (unrelated to linear algebra): load an image, convert it to grayscale, and extract the RGB layers